In [2]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as inputData
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
import time
from datetime import timedelta

In [4]:
data = inputData.read_data_sets('data', one_hot=True)

Extracting data\train-images-idx3-ubyte.gz
Extracting data\train-labels-idx1-ubyte.gz
Extracting data\t10k-images-idx3-ubyte.gz
Extracting data\t10k-labels-idx1-ubyte.gz


In [5]:
def plotNumbers(imset, valset, dimen):
    fig, axis = plt.subplots(dimen[0],dimen[1])

    for pos, sub in enumerate(axis.flat):
        sub.imshow(imset[pos].reshape((28,28)))
        sub.set_xticks([])
        sub.set_yticks([])
        sub.set_xlabel('pred:%r' %valset[pos])
    plt.show()

In [14]:
image_size = 28
input_size = 784
LRF_size = 5
cn1_filters = 12
cn2_filters = 20
fc_layer = 100
output_layer = 10


X = tf.placeholder(tf.float32, [None, input_size])
X_for_conv = tf.reshape(X, [-1, image_size, image_size, 1])
Y_ori = tf.placeholder(tf.float32, shape=[None, output_layer])
is_dropOut = tf.placeholder(tf.bool)

Wcn1 = tf.Variable(initial_value=tf.truncated_normal(shape=[LRF_size, LRF_size, 1, cn1_filters], stddev=0.1))
bcn1 = tf.Variable(initial_value=tf.ones(shape=[cn1_filters])/10)

cn1 = tf.nn.conv2d(X_for_conv, Wcn1, strides=[1, 1, 1, 1], padding='SAME') + bcn1
cn1_relu = tf.nn.relu(cn1)
if is_dropOut is True:
    cn1_relu = tf.nn.dropout(cn1_relu, keep_prob=0.75) 

print('input is             %a' %X_for_conv)
print('weight is            %a' %Wcn1)
print('bias is              %a' %bcn1)
print('conv out, cn1 is     %a' %cn1)
print('conv relu, cn1_relu  %a' %cn1_relu)

input is             <tf.Tensor 'Reshape_8:0' shape=(?, 28, 28, 1) dtype=float32>
weight is            <tf.Variable 'Variable_10:0' shape=(5, 5, 1, 12) dtype=float32_ref>
bias is              <tf.Variable 'Variable_11:0' shape=(12,) dtype=float32_ref>
conv out, cn1 is     <tf.Tensor 'add_5:0' shape=(?, 28, 28, 12) dtype=float32>
conv relu, cn1_relu  <tf.Tensor 'Relu_5:0' shape=(?, 28, 28, 12) dtype=float32>


In [15]:
Wcn2 = tf.Variable(initial_value=tf.truncated_normal(shape=[LRF_size, LRF_size, cn1_filters, cn2_filters], stddev=0.1))
bcn2 = tf.Variable(initial_value=tf.ones(cn2_filters)/10)

cn2 = tf.nn.conv2d(cn1_relu, Wcn2, strides=[1, 1, 1, 1], padding='SAME') + bcn2
cn2_relu = tf.nn.relu(cn2)
if is_dropOut is True:
    cn2_relu = tf.nn.dropout(cn2_relu, keep_prob=0.75)

print('weight is            %a' %Wcn2)
print('bias is              %a' %bcn2)
print('conv out, cn2 is     %a' %cn2)
print('conv relu, cn2_relu  %a' %cn2_relu)

weight is            <tf.Variable 'Variable_12:0' shape=(5, 5, 12, 20) dtype=float32_ref>
bias is              <tf.Variable 'Variable_13:0' shape=(20,) dtype=float32_ref>
conv out, cn2 is     <tf.Tensor 'add_6:0' shape=(?, 28, 28, 20) dtype=float32>
conv relu, cn2_relu  <tf.Tensor 'Relu_6:0' shape=(?, 28, 28, 20) dtype=float32>


In [16]:
fully_connected_input_size = cn2_relu.shape[1:4].num_elements()
cn2_reshaped = tf.reshape(cn2_relu, shape=[-1, fully_connected_input_size])

Wfc = tf.Variable(initial_value=tf.truncated_normal(shape=[fully_connected_input_size, fc_layer], stddev=0.1)) 
bfc = tf.Variable(initial_value=tf.ones(fc_layer)/10)

print('cn2_reshaped is      %a' %cn2_reshaped)
print('weight is            %a' %Wfc)
print('bias is              %a' %bfc)

logits_fc = tf.matmul(cn2_reshaped, Wfc) + bfc
if is_dropOut is True:
    logits_fc = tf.nn.dropout(logits_fc, keep_prob=0.75)

print('logits_fc is          %a' %logits_fc)

cn2_reshaped is      <tf.Tensor 'Reshape_9:0' shape=(?, 15680) dtype=float32>
weight is            <tf.Variable 'Variable_14:0' shape=(15680, 100) dtype=float32_ref>
bias is              <tf.Variable 'Variable_15:0' shape=(100,) dtype=float32_ref>
logits_fc is          <tf.Tensor 'add_7:0' shape=(?, 100) dtype=float32>


In [9]:
WL = tf.Variable(initial_value=tf.truncated_normal(shape=[fc_layer, output_layer], stddev=0.1)) 
bL = tf.Variable(initial_value=tf.ones(output_layer)/10)

print('weight is            %a' %WL)
print('bias is              %a' %bL)

logits_L = tf.matmul(logits_fc, WL) + bL
Y_pred = tf.nn.softmax(logits_L)

print('logits_L is          %a' %logits_L)

weight is            <tf.Variable 'Variable_6:0' shape=(100, 10) dtype=float32_ref>
bias is              <tf.Variable 'Variable_7:0' shape=(10,) dtype=float32_ref>
logits_L is          <tf.Tensor 'add_3:0' shape=(?, 10) dtype=float32>


In [11]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits_L, labels=Y_ori)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=0.003).minimize(cost)

correct_prediction = tf.equal(tf.argmax(Y_pred, axis=1), tf.argmax(Y_ori, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [133]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

accu_list = []
cost_list = []
iteration = 50
acc_check_list = np.arange(0, iteration, int(iteration/50))

start_time = time.time()
for val in range(iteration):
    next_batch_x, next_batch_y = data.train.next_batch(100)
    sess.run(optimizer, feed_dict={X: next_batch_x, Y_ori: next_batch_y, is_dropOut: True})

    if val in acc_check_list:
        accu, cost_ = sess.run((accuracy, cost), feed_dict={X: data.test.images, Y_ori: data.test.labels, is_dropOut: False})
        accu_list.append(accu)
        cost_list.append(cost_)
    
print(accu_list)

end_time = time.time()
print('total time taken is %s' %str(timedelta(seconds=int(round(end_time-start_time)))))

[0.0779, 0.0904, 0.1809, 0.237, 0.224, 0.2546, 0.2656, 0.2381, 0.2602, 0.2892]
[5.4047694, 5.7076063, 3.8730843, 2.9990046, 2.39899, 2.154399, 2.0794637, 2.051934, 2.0272703, 1.982882]


In [ ]:
plt.plot(acc_check_list, accu_list[:])
#plt.axis([0,iteration,0.88,0.99])

plt.plot(acc_check_list, cost_list[:])
plt.show()

In [ ]:
plt.plot(acc_check_list, accu_list[:])
plt.axis([0,iteration,0.88,0.99])
plt.show()

In [ ]:
li = ['a', 'a', 'b','b','b', 'b','c','d','d','d','d']
#one option
max(li,key=li.count)

#second option
from collections import Counter
data = Counter(li)
data.most_common(1)[0][0]